In [3]:
import numpy as np
import pandas as pd


### Exercise 2

In [12]:
def eclat(db, minsup):
    start_time=datetime.now()
    time_format="%H:%M:%S:%f"
    temporal_items_list = []
    for t in db:
        temporal_items_list.extend(t)
    temporal_items_list = list(sorted(set(temporal_items_list)))
    item_list = temporal_items_list.copy()
    item_list_transactions = {item: [] for item in temporal_items_list}
    for transaction_index, t in enumerate(db):
        for transaction_item in t:
            item_list_transactions[(transaction_item)].append(transaction_index)
    itemset_supports = [([item], item_list_transactions[item]) for item in item_list]
    items_temporal=itemset_supports.copy()
    temporal_items_list = item_list.copy()
    
    for itemset in itemset_supports:
        if len(itemset[1]) < minsup:
            temporal_items_list.remove(itemset[0][0])
            items_temporal.remove(itemset)
    itemset_supports=items_temporal.copy()
    item_list = temporal_items_list.copy()
    
    final_supports_list=supportsNextLevel(itemset_supports,itemset_supports, item_list, minsup)
    final_supports=[(item,len(transactions)) for item,transactions in final_supports_list]
    final_time=datetime.now()
    execution_time=(final_time-start_time)
    print(f'Runtime: {execution_time}\n    Start Time: {start_time.strftime(time_format)}\n    Finish Time: {final_time.strftime(time_format)}')
    return final_supports

    
def supportsNextLevel(total_supports, supports, items, minsup, first_time=True):
    for index, itemset in enumerate(items[:-1]):
        level_supports = []
        itemset_derivate_list = []
        items_to_combine = items[index+1:]
        for other_itemset in items_to_combine:
            if first_time:
                new_itemset_support = findSupport(supports, [itemset], [other_itemset])
                new_itemset_list = sorted(list(set([itemset, other_itemset])))
            else:
                new_list = []
                new_list.extend(itemset)
                new_list.extend(other_itemset)
                new_itemset_support = findSupport(supports, itemset, other_itemset)
                new_itemset_list = sorted(list(set(new_list)))
            if len(new_itemset_support) >= minsup:
                level_supports.append((new_itemset_list, new_itemset_support))
                itemset_derivate_list.append(new_itemset_list)   
        total_supports.extend(level_supports)
        if len(level_supports) > 1 and level_supports:
            supportsNextLevel(total_supports, level_supports, itemset_derivate_list, minsup, False)
    if first_time:
        return total_supports


def findSupport(supports, first_itemset, second_itemset):
    for support in supports:
        if first_itemset == support[0]:
            first_support = support[1]
        if second_itemset == support[0]:
            second_support = support[1]
    return list(set(first_support) & set(second_support))

def getStrongRulesFromFrequentSets(fsets, minconf):
   strong_rules = []
   fsets_sets = [fset[0] for fset in fsets]
   fsets_supports = [fset[1] for fset in fsets]

   for i, set in enumerate(fsets_sets):
        if len(set) >= 2:
           A = subsets_non_empty(set)
           while A:
               X = max(A, key=lambda x: fsets_supports[fsets_sets.index(x)])
               A.remove(X)
               index = fsets_sets.index(X)
               sup_z = fsets_supports[i]
               sup_x = fsets_supports[index]
               conf = sup_z / sup_x
               if conf >= minconf:
                   Y = set.copy()
                   for x in X:
                       Y.remove(x)
                   strong_rules.append((X, Y, fsets_supports[i], conf))
               else: 
                  for w in X:
                     if w in A:
                        A.remove(x)                
   return strong_rules

def subsets_non_empty(Z):
   all_subsets = chain.from_iterable(combinations(Z,r) for r in range(1, len(Z) + 1)) 
   non_empty_subsets = [sorted(subset) for subset in all_subsets if subset]
   non_empty_subsets.sort(key=lambda x: (len(x), x))
   non_empty_subsets.pop(-1)
   return non_empty_subsets

def getStrongRulesForDatabase(db, minsup, minconf):
  fsets = eclat(db, minsup)
  strong_rules = getStrongRulesFromFrequentSets(fsets, minconf)
  return strong_rules

### 2.1

In [29]:
def sup(D, X, Y=None):
    supp = 0
    if Y is None:
        for transaction in D:
            if all(item in transaction for item in X):
                supp += 1
    else:
        for transaction in D:
            if all(item in transaction for item in X) and all(item in transaction for item in Y):
                supp += 1
    return supp

def conf(D, X, Y):
    return sup(D, X, Y) / sup(D, X)

def lift(D, X, Y=None):
    if Y is None:
        Y = X.copy()
    supports = [sum(all(item in transaction for item in items) for transaction in D) for items in (X, Y, X + Y)]
    if 0 in (r_support := [sup / len(D) for sup in supports]):
        return None
    return r_support[2] / (r_support[0] * r_support[1])

def leverage(D, X, Y):
    supports = [sum(all(item in transaction for item in items) for transaction in D) for items in (X, Y, X + Y)]
    final_leverage = supports[2] / len(D) - (supports[0] / len(D)) * (supports[1] / len(D))
    return final_leverage

def jaccard(D, X, Y):
    return sup(D, X, Y) / (sup(D, X) + sup(D, Y) - sup(D, X, Y))

def conviction(D, X, Y):
    support_y = sum(all(item in transaction for item in Y) for transaction in D) / len(D)
    confidence_xy = conf(D, X, Y)
    if confidence_xy >= 1:
        return None
    return (1 - support_y) / (1 - confidence_xy)

def oddsRatio(D, X, Y):
    supports = {(False, False): 0, (False, True): 0, (True, False): 0, (True, True): 0}
    for transaction in D:
        supports[(all(item in transaction for item in X), all(item in transaction for item in Y))] += 1
    final_odds_ratio = ((supports[(True, True)] + 1) * (supports[(False, False)] + 1)) / ((supports[(False, True)] + 1) * (supports[(True, False)] + 1))
    return final_odds_ratio

def imp(D, X, Y):
    if len(X) < 2:
        return 0
    conf_xy = conf(D, X, Y)
    w_conf = [conf(D, W, Y) for W in X]
    return conf_xy - max(w_conf) if w_conf else 0

def getRuleMetric(D,X,Y,metric):
    if metric == "sup":
        return sup(D,X,Y)
    elif metric == "conf":
        return conf(D,X,Y)
    elif metric == "lift":
        return lift(D,X,Y)
    elif metric == "leverage":
        return leverage(D,X,Y)
    elif metric == "jaccard":
        return jaccard(D,X,Y)
    elif metric == "conviction":
        return conviction(D,X,Y)
    elif metric == "oddsratio":
        return oddsRatio(D,X,Y)
    elif metric == "imp":
        return imp(D,X,Y)
    else:
        return "metrica invalida"
    


### 2.2

In [15]:
def filterProductiveRules(D, R):
    productive_rules = []
    for rule in R:
        X, Y, sup_XY, conf_XY = rule
        if conf_XY > 0:
            productive_rules.append(rule)
    return productive_rules

### 2.3